In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np

# Part I - Toronto_Neighborhoods

### To scrape the data of the neighborhoods in Toronto from Wikipedia into dataframe type

In [2]:
column_name = ['PostalCode', 'Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns = column_name)

In [3]:
url_str = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = urlopen(url_str)
soup = BeautifulSoup(url, 'html.parser')

In [4]:
table = soup.find('table', attrs = {'class': 'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [c.text.strip() for c in cols]
    
    if len(cols) > 1:
        neighborhoods = neighborhoods.append({'PostalCode': cols[0], 'Borough' : cols[1], 'Neighborhood': cols[2]}, ignore_index= True)


In [5]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore the rows with a borough that is "not assigned"

In [6]:
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned']

In [7]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### merge the values of "Neighborhood" based on the "PostalCode" and "Borough"

In [8]:
neighborhoods = neighborhoods.groupby(['PostalCode', 'Borough']).agg({'Neighborhood' : ','.join}).reset_index()

### If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [9]:
neighborhoods['Neighborhood'] = np.where(neighborhoods['Neighborhood'] == 'Not assigned', neighborhoods['Borough'], neighborhoods['Neighborhood'])

In [11]:
neighborhoods.shape

(103, 3)

In [15]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Part II - Data Merge

### input latitude/longitude data and merge them

In [16]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [17]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df = neighborhoods.merge(df_geo, how = 'left', left_on= 'PostalCode', right_on = 'Postal Code')

In [23]:
df.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353


In [22]:
#remove irrelevant columns
df.drop( 'Postal Code', axis= 1, inplace= True)